In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Define constants
image_size = (128, 128)  # Adjust the size based on your dataset
batch_size = 16

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, 
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, vertical_flip =True)
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/bone-fracture-dataset/BoneFractureDataset/training',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/bone-fracture-dataset/BoneFractureDataset/testing',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [4]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Nadam, Adamax
from keras.layers import Input, Conv2D, MaxPooling2D,GlobalAveragePooling2D,LSTM
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("Bone_fracture_classifier_model.keras", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', verbose=1),
]

# Compile the model
model.compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [5]:
# Train the model
history = model.fit(
    train_generator, 
    epochs=10,
    validation_data=test_generator,
    callbacks = [callbacks], verbose = 1
   )

Epoch 1/10
554/554 [==============================] - 189s 336ms/step - loss: 0.6886 - accuracy: 0.5332 - val_loss: 0.6469 - val_accuracy: 0.6000
Epoch 2/10
554/554 [==============================] - 156s 280ms/step - loss: 0.6804 - accuracy: 0.5490 - val_loss: 0.5457 - val_accuracy: 0.6700
Epoch 3/10
554/554 [==============================] - 158s 285ms/step - loss: 0.6703 - accuracy: 0.5736 - val_loss: 0.8300 - val_accuracy: 0.6000
Epoch 4/10
554/554 [==============================] - 156s 281ms/step - loss: 0.6514 - accuracy: 0.6066 - val_loss: 0.5257 - val_accuracy: 0.7467
Epoch 5/10
554/554 [==============================] - 158s 284ms/step - loss: 0.6299 - accuracy: 0.6303 - val_loss: 0.7412 - val_accuracy: 0.5533
Epoch 6/10
554/554 [==============================] - 156s 282ms/step - loss: 0.6204 - accuracy: 0.6443 - val_loss: 0.8460 - val_accuracy: 0.5300
Epoch 7/10
554/554 [==============================] - 155s 280ms/step - loss: 0.6043 - accuracy: 0.6626 - val_loss: 0.6942 -

In [ ]:
# Evaluate the model
evaluation = model.evaluate(test_generator)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

In [ ]:
# Make predictions on new data
predictions = model.predict(test_generator)

In [ ]:
# Convert predictions to binary labels
predicted_labels = (predictions > 0.5).astype(int)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Get true labels
true_labels = test_generator.classes

# Generate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
# Generate classification report
class_report = classification_report(true_labels, predicted_labels, target_names=['not_fractured', 'fractured'])
print("Classification Report:")
print(class_report)

In [ ]:
model.save("my_model.h5")

In [ ]:
import matplotlib.pyplot as plt
import random

# Visualize random predictions
test_images, true_labels = next(test_generator)
predicted_labels = (model.predict(test_images) > 0.5).astype(int)

# Plot some sample images with true and predicted labels
plt.figure(figsize=(12, 8))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(test_images[i])
    plt.title(f"True: {true_labels[i]}, Predicted: {predicted_labels[i]}")
    plt.axis('off')
plt.show()

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Shallow Neural Network Baseline

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Get the image dimensions
image_shape = (128, 128, 3)
batch_size2 = 64

# Build a shallow neural network model
shallow_model = Sequential([
    Flatten(input_shape=image_shape),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile and train the shallow model
shallow_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Create a data generator for the training data
train_generator2 = train_datagen.flow_from_directory(
    '/kaggle/input/bone-fracture-dataset/BoneFractureDataset/training',
    target_size=image_size,
    batch_size=batch_size2,
    class_mode='binary'
)

# Train the shallow model
shallow_model.fit(train_generator2, epochs=5, validation_data=test_generator)

# Evaluate the shallow model
shallow_accuracy = shallow_model.evaluate(test_generator)[1]

# Get the predictions
y_pred = (shallow_model.predict(test_generator) > 0.5).astype(int)

# Get the true labels
y_test = test_generator.classes

# Calculate the classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
shallow_report = classification_report(y_test, y_pred, target_names=['not_fractured', 'fractured'])
shallow_conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Shallow Neural Network Accuracy: {shallow_accuracy * 100:.2f}%")
print("Shallow Neural Network Classification Report:\n", shallow_report)
print("Shallow Neural Network Confusion Matrix:\n", shallow_conf_matrix)

# Trying transfer learning

# 1 -> VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report

# Define constants
image_size = (128, 128)
batch_size = 32
train_generator3 = train_datagen.flow_from_directory(
    '/kaggle/input/bone-fracture-dataset/BoneFractureDataset/training',
    target_size=image_size,
    batch_size=32,
    class_mode='binary'
)
# Load pre-trained VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Freeze the convolutional base
for layer in vgg_model.layers:
    layer.trainable = False

# Add custom classifier on top
custom_model = Sequential([
    vgg_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the custom model
custom_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the custom model
history = custom_model.fit(train_generator3, epochs=10, validation_data=test_generator)

# Evaluate the custom model
evaluation_VGG16 = custom_model.evaluate(test_generator)
print("Test Accuracy:", evaluation_VGG16[1])


In [ ]:
# Make predictions on test data
predictions_VGG16 = custom_model.predict(test_generator)
predicted_labels_VGG16 = (predictions_VGG16 > 0.5).astype(int)
# Get true labels
true_labels_VGG16 = test_generator.classes

# Generate confusion matrix
conf_matrix_VGG16 = confusion_matrix(true_labels_VGG16, predicted_labels_VGG16)
print("Confusion Matrix:")
print(conf_matrix_VGG16)

# Generate classification report
class_report_VGG16 = classification_report(true_labels_VGG16, predicted_labels_VGG16, target_names=['not_fractured', 'fractured'])
print("Classification Report:")
print(class_report_VGG16)

# 2 -> DenseNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report

# Define constants
image_size = (128, 128)
batch_size = 32

# Load pre-trained DenseNet121 model
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Freeze the convolutional base
for layer in densenet_model.layers:
    layer.trainable = False

# Add custom classifier on top
custom_model_densenet = Sequential([
    densenet_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the custom model
custom_model_densenet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the custom model
history_densenet = custom_model_densenet.fit(train_generator3, epochs=10, validation_data=test_generator)

# Evaluate the custom model
evaluation_densenet = custom_model_densenet.evaluate(test_generator)
print("Test Accuracy (DenseNet):", evaluation_densenet[1])

# Make predictions on test data
predictions_densenet = custom_model_densenet.predict(test_generator)
predicted_labels_densenet = (predictions_densenet > 0.5).astype(int)

# Get true labels
true_labels_densenet = test_generator.classes

# Generate confusion matrix
conf_matrix_densenet = confusion_matrix(true_labels_densenet, predicted_labels_densenet)
print("Confusion Matrix (DenseNet):")
print(conf_matrix_densenet)

# Generate classification report
class_report_densenet = classification_report(true_labels_densenet, predicted_labels_densenet, target_names=['not_fractured', 'fractured'])
print("Classification Report (DenseNet):")
print(class_report_densenet)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Model performance data
data = {
    'Model': ['Custom CNN', 'Shallow NN', 'VGG16', 'DenseNet121'],
    'Test Accuracy': [evaluation[1], shallow_accuracy, evaluation_VGG16[1], evaluation_densenet[1]]
}

# Create DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

# Plot Test Accuracy Comparison
plt.figure(figsize=(10, 6))
plt.barh(df['Model'], df['Test Accuracy'], color=['skyblue', 'lightgreen', 'orange', 'purple'])
plt.xlabel('Test Accuracy')
plt.ylabel('Model')
plt.title('Test Accuracy Comparison of Bone Fracture Classification Models')
plt.xlim(0, 1)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the model names
model_names = ["Custom CNN", "Shallow NN", "VGG16", "DenseNet121"]

# Define the evaluation metrics
accuracy = [evaluation[1], shallow_accuracy, evaluation_VGG16[1], evaluation_densenet[1]]
precision = [precision_score(true_labels, predicted_labels), precision_score(y_test, y_pred),
             precision_score(true_labels_VGG16, predicted_labels_VGG16), precision_score(true_labels_densenet, predicted_labels_densenet)]
recall = [recall_score(true_labels, predicted_labels), recall_score(y_test, y_pred),
          recall_score(true_labels_VGG16, predicted_labels_VGG16), recall_score(true_labels_densenet, predicted_labels_densenet)]
f1 = [f1_score(true_labels, predicted_labels), f1_score(y_test, y_pred),
      f1_score(true_labels_VGG16, predicted_labels_VGG16), f1_score(true_labels_densenet, predicted_labels_densenet)]

# Create a DataFrame
df = pd.DataFrame({
    "Model": model_names,
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1-score": f1
})

# Print the DataFrame
print("Comparison of Bone Fracture Classification Models:")
print(df)

import numpy as np

# Function to generate random colors
def generate_random_colors(num_colors):
    return [plt.cm.tab10(i) for i in np.random.rand(num_colors)]



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define custom colors for each plot
colors_accuracy = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
colors_precision = ['#9467bd', '#8c564b', '#e377c2', '#7f7f7f']
colors_recall = ['#bcbd22', '#17becf', '#1f77b4', '#d62728']
colors_f1 = ['#2ca02c', '#ff7f0e', '#d62728', '#9467bd']

# Plotting
plt.figure(figsize=(12, 8))

# Plot accuracy
plt.subplot(2, 2, 1)
plt.bar(model_names, accuracy, color=colors_accuracy)
plt.title('Test Accuracy Comparison')
plt.ylabel('Accuracy')

# Plot precision
plt.subplot(2, 2, 2)
plt.bar(model_names, precision, color=colors_precision)
plt.title('Precision Comparison')
plt.ylabel('Precision')

# Plot recall
plt.subplot(2, 2, 3)
plt.bar(model_names, recall, color=colors_recall)
plt.title('Recall Comparison')
plt.ylabel('Recall')

# Plot F1-score
plt.subplot(2, 2, 4)
plt.bar(model_names, f1, color=colors_f1)
plt.title('F1-score Comparison')
plt.ylabel('F1-score')

plt.tight_layout()
plt.show()
